In [115]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,to_timestamp

In [117]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [118]:
spark.version

'3.5.5'

In [119]:
df = spark.read \
    .option("header", "true") \
    .parquet('yellow_tripdata_2024-10.parquet')

In [120]:
df = df.repartition(4)

In [123]:
df.write.parquet('yellow/2024/10', mode='overwrite')

In [124]:
df = spark.read.parquet('yellow/2024/10')

In [125]:
df = df.withColumn('pickup_time',to_timestamp(col('tpep_pickup_datetime'))) \
    .withColumn('dropoff_time',to_timestamp(col('tpep_dropoff_datetime'))) \
    .withColumn('timeDiff',col('dropoff_time').cast('long') - col('pickup_time').cast('long')) \
    .withColumn('trip_length_hours',(col('timeDiff')/3600).cast('integer'))

In [126]:
df.registerTempTable('tripdata')

In [127]:
df_count = spark.sql("""
SELECT 
    COUNT(1) AS trip_count
FROM 
    tripdata
WHERE 
    date_trunc('day',tpep_pickup_datetime) = '2024-10-15 00:00:00'
""")

In [128]:
df_count.show()

+----------+
|trip_count|
+----------+
|    128893|
+----------+



In [129]:
df_length = spark.sql("""
SELECT 
    trip_length_hours
FROM 
    tripdata
ORDER BY trip_length_hours DESC
""")

In [132]:
df_length.show(1)

+-----------------+
|trip_length_hours|
+-----------------+
|              162|
+-----------------+
only showing top 1 row



In [133]:
df_zones = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [136]:
df_join = df.join(df_zones, df.PULocationID == df_zones.LocationID)

In [139]:
df_join.registerTempTable('zonedata')

In [141]:
df_zone_data = spark.sql("""
SELECT 
    zone,
    count(1) AS trip_count
FROM 
    zonedata
GROUP BY zone
ORDER BY trip_count ASC
""")

In [144]:
df_zone_data.show(1)

+--------------------+----------+
|                zone|trip_count|
+--------------------+----------+
|Governor's Island...|         1|
+--------------------+----------+
only showing top 1 row

